In [1]:
import numpy as np
import scipy as sp
import suite2p as s2p
import TwoPUtils
import os
from matplotlib import pyplot as plt

%matplotlib inline

%load_ext autoreload
%autoreload 2

In [7]:

write_folder = os.path.join('/mnt','BigDisk','2P_scratch')


mouse = '4467975.1'

file_list = ({'date': '28_09_2020', 'scene':'YMaze_LNovel','session':1,'scan':3},
             {'date': '28_09_2020', 'scene':'YMaze_LNovel','session':6,'scan':14},
             {'date': '29_09_2020', 'scene':'YMaze_LNovel','session':1,'scan':3},
             {'date': '30_09_2020', 'scene':'YMaze_LNovel','session':1,'scan':4},
             {'date': '01_10_2020', 'scene':'YMaze_LNovel','session':1,'scan':2},
             {'date': '02_10_2020', 'scene':'YMaze_LNovel','session':1,'scan':3},
             {'date': '03_10_2020', 'scene':'YMaze_LNovel','session':1,'scan':3},
             {'date': '04_10_2020', 'scene':'YMaze_LNovel','session':1,'scan':3},
             {'date': '04_10_2020', 'scene':'YMaze_RewardReversal','session':2,'scan':10},
             {'date': '05_10_2020', 'scene':'YMaze_RewardReversal','session':1,'scan':3},
             {'date': '06_10_2020', 'scene':'YMaze_RewardReversal','session':1,'scan':2},
             {'date': '06_10_2020', 'scene':'YMaze_LNovel','session':1,'scan':5},
             )



In [8]:
def check_and_make_h5s(file_list, runh5=True):
    h5list, framerates = [], []
    for f in file_list:
        print(f)
        scan_str = "%s_%03d_%03d" % (f['scene'],f['session'],f['scan'])
        try:
            source_folder  = os.path.join('/media','mplitt','Backup Plus','2P_Data')
            source_stem = os.path.join(source_folder,mouse,f['date'],f['scene'],scan_str)
            info = TwoPUtils.scanner_tools.sbx_utils.loadmat(source_stem+'.mat')
        except:
            source_folder  = os.path.join('/media','mplitt','Backup Plus1','2P_Data')
            source_stem = os.path.join(source_folder,mouse,f['date'],f['scene'],scan_str)
            info = TwoPUtils.scanner_tools.sbx_utils.loadmat(source_stem+'.mat')
        
        file_stem = os.path.join(write_folder,mouse,f['date'],f['scene'],scan_str)
        h5path = os.path.join(file_stem,scan_str+".h5")
        if not os.path.exists(h5path) and runh5:
            _ = TwoPUtils.scanner_tools.sbx_utils.sbx2h5(source_stem,output_name=h5path,force_2chan=True)

        h5list.append(file_stem)
        framerates.append(info['frame_rate'])
    print(h5list)
    return h5list, framerates

In [19]:
ops = np.load(os.path.join(write_folder,mouse,"YMazeAll","suite2p","plane0","ops.npy"),allow_pickle=True).all()

In [21]:
print(ops.keys())

dict_keys(['suite2p_version', 'look_one_level_down', 'fast_disk', 'delete_bin', 'mesoscan', 'bruker', 'h5py', 'h5py_key', 'save_path0', 'save_folder', 'subfolders', 'move_bin', 'nplanes', 'nchannels', 'functional_chan', 'tau', 'fs', 'force_sktiff', 'frames_include', 'multiplane_parallel', 'preclassify', 'save_mat', 'save_NWB', 'combined', 'aspect', 'do_bidiphase', 'bidiphase', 'bidi_corrected', 'do_registration', 'two_step_registration', 'keep_movie_raw', 'nimg_init', 'batch_size', 'maxregshift', 'align_by_chan', 'reg_tif', 'reg_tif_chan2', 'subpixel', 'smooth_sigma_time', 'smooth_sigma', 'th_badframes', 'norm_frames', 'force_refImg', 'pad_fft', 'nonrigid', 'block_size', 'snr_thresh', 'maxregshiftNR', '1Preg', 'spatial_hp', 'spatial_hp_reg', 'spatial_hp_detect', 'pre_smooth', 'spatial_taper', 'roidetect', 'spikedetect', 'anatomical_only', 'sparse_mode', 'diameter', 'spatial_scale', 'connected', 'nbinned', 'max_iterations', 'threshold_scaling', 'max_overlap', 'high_pass', 'use_builtin_c

In [27]:
print(ops['save_path0'])
print(ops['save_path'])
print(ops['ops_path'])
print(ops['reg_file'])
print(ops['reg_file_chan2'])
ops['save_path0'] = os.path.join(write_folder,mouse,"YMazeAll")
ops['save_path'] = os.path.join(write_folder,mouse,"YMazeAll","suite2p","plane0")
ops['ops_path'] = os.path.join(write_folder,mouse,"YMazeAll","suite2p","plane0","ops.npy")
ops['reg_file'] = os.path.join(write_folder,mouse,"YMazeAll","suite2p","plane0","data.bin")
ops['reg_file_chan2'] = os.path.join(write_folder,mouse,"YMazeAll","suite2p","plane0","data_chan2.bin")
np.save(ops['ops_path'],ops)

/media/mplitt/BigDisk/2P_scratch/4467975.1/YMazeAll
/media/mplitt/BigDisk/2P_scratch/4467975.1/YMazeAll/suite2p/plane0
/media/mplitt/BigDisk/2P_scratch/4467975.1/YMazeAll/suite2p/plane0/ops.npy
/media/mplitt/BigDisk/2P_scratch/4467975.1/YMazeAll/suite2p/plane0/data.bin
/media/mplitt/BigDisk/2P_scratch/4467975.1/YMazeAll/suite2p/plane0/data_chan2.bin


In [28]:
h5list_all, frs_all = check_and_make_h5s(file_list)
ops = TwoPUtils.s2p.set_ops(d={'data_path': h5list_all,
                               'save_path0': os.path.join(write_folder,mouse,"YMazeAll"),
                                'fast_disk':[],
                                'move_bin':True,
                                'two_step_registration':True,
                                'maxregshiftNR':10,
                                'nchannels':2,
                                'tau':.7,
                                'functional_chan':1,
                                'fs':frs_all[0],
                                'roidetect':True,
                                'input_format':"h5",
                                'h5py_key':'data',
                                'nchannels':2,
                                'saveNWB':True,
                                'threshold_scaling':.75,
                                'chan2_thresh':.01})
ops=s2p.run_s2p(ops=ops)

{'date': '28_09_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 3}
{'date': '28_09_2020', 'scene': 'YMaze_LNovel', 'session': 6, 'scan': 14}
{'date': '29_09_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 3}
{'date': '30_09_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 4}
{'date': '01_10_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 2}
{'date': '02_10_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 3}
{'date': '03_10_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 3}
{'date': '04_10_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 3}
{'date': '04_10_2020', 'scene': 'YMaze_RewardReversal', 'session': 2, 'scan': 10}
{'date': '05_10_2020', 'scene': 'YMaze_RewardReversal', 'session': 1, 'scan': 3}
{'date': '06_10_2020', 'scene': 'YMaze_RewardReversal', 'session': 1, 'scan': 2}
{'date': '06_10_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 5}
['/mnt/BigDisk/2P_scratch/4467975.1/28_09_2020/YMaze_LNovel/YMaze_LNovel_001_003', '/mnt/BigDisk/2

In [10]:
_file_list = ({'date': '29_09_2020', 'scene':'YMaze_LNovel','session':1,'scan':3},
             {'date': '30_09_2020', 'scene':'YMaze_LNovel','session':1,'scan':4},
             {'date': '01_10_2020', 'scene':'YMaze_LNovel','session':1,'scan':2},
             {'date': '02_10_2020', 'scene':'YMaze_LNovel','session':1,'scan':3},
             #{'date': '03_10_2020', 'scene':'YMaze_LNovel','session':1,'scan':3},
             {'date': '05_10_2020', 'scene':'YMaze_RewardReversal','session':1,'scan':3},
             )


for f in _file_list:
    
    _h5list,_frs = check_and_make_h5s([f,],runh5=True)
    ops_dir = os.path.join(_h5list[0],'suite2p','plane0','ops.npy')
    s2p_dir = os.path.join(_h5list[0],'suite2p','plane0')
    if os.path.exists(ops_dir):
        ops = np.load(os.path.join(s2p_dir,"ops.npy"),allow_pickle=True).all()
        ops['reg_file'] = os.path.join(s2p_dir,"data.bin")
        ops['reg_file_chan2'] = os.path.join(s2p_dir,"data_chan2.bin")
        np.save(ops_dir,ops)

    # if ops file exists
        # update paths to be local paths
    ops = TwoPUtils.s2p.set_ops(d={'data_path': _h5list,
                                   'save_path0': _h5list[0],
                                    'fast_disk':[],
                                    'move_bin':True,
                                    'two_step_registration':True,
                                    'maxregshiftNR':10,
                                    'nchannels':2,
                                    'tau':.7,
                                    'functional_chan':1,
                                    'fs':_frs[0],
                                    'roidetect':True,
                                    'input_format':"h5",
                                    'h5py_key':'data',
                                    'nchannels':2,
                                    'threshold_scaling':1,
                                    'chan2_thresh':.01})
    ops=s2p.run_s2p(ops=ops)
    !rm {_h5list[0]}/*.h5 

{'date': '29_09_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 3}
0 1000
1000 2000
2000 3000
3000 4000
4000 5000
5000 6000
6000 7000
7000 8000
8000 9000
9000 10000
10000 11000
11000 12000
12000 13000
13000 14000
14000 15000
15000 16000
16000 17000
17000 18000
18000 19000
19000 20000
20000 21000
21000 22000
22000 23000
23000 24000
24000 25000
25000 26000
26000 27000
27000 28000
28000 29000
29000 30000
30000 31000
31000 32000
32000 33000
33000 34000
34000 35000
35000 36000
36000 37000
37000 38000
38000 38344
['/mnt/BigDisk/2P_scratch/4467975.1/29_09_2020/YMaze_LNovel/YMaze_LNovel_001_003']
{}
FOUND BINARIES AND OPS IN ['/mnt/BigDisk/2P_scratch/4467975.1/29_09_2020/YMaze_LNovel/YMaze_LNovel_001_003/suite2p/plane0/ops.npy']
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
NOTE: not running registration, plane already registered
NOTE: applying default /home/mplitt/.suite2p/classifiers/classifier_user.npy
----------- ROI DETECTION
Binning movie in chunks of length 11
Binned movie 

In [9]:
ops['reg_file_chan2']

'/media/mplitt/BigDisk/2P_scratch/4467975.1/29_09_2020/YMaze_LNovel/YMaze_LNovel_001_003/suite2p/plane0/data_chan2.bin'

In [18]:
_file_list = ({'date': '28_09_2020', 'scene':'YMaze_LNovel','session':1,'scan':3},
             {'date': '28_09_2020', 'scene':'YMaze_LNovel','session':6,'scan':14},)

_h5list,_frs = check_and_make_h5s(_file_list,runh5=False)
# ops_dir = os.path.join(write_folder,mouse,"28_09_2020","combined","suite2p","plane0","ops.npy")
# ops = np.load(ops_dir, allow_pickle=True).all()
# ops['reg_file'] = os.path.join(s2p_dir,"data.bin")
# ops['reg_file_chan2'] = os.path.join(s2p_dir,"data_chan2.bin")
# np.save(ops_dir,ops)


ops = TwoPUtils.s2p.set_ops(d={'data_path': _h5list,
                               'save_path0': os.path.join(write_folder,mouse,"28_09_2020","combined"),
                                'fast_disk':[],
                                'move_bin':True,
                                'two_step_registration':True,
                                'maxregshiftNR':10,
                                'nchannels':2,
                                'tau':.7,
                                'functional_chan':1,
                                'fs':_frs[0],
                                'roidetect':True,
                                'input_format':"h5",
                                'h5py_key':'data',
                                'nchannels':2,
                                'threshold_scaling':1.5,
                                'anatomical_only':True})
ops=s2p.run_s2p(ops=ops)

{'date': '28_09_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 3}
{'date': '28_09_2020', 'scene': 'YMaze_LNovel', 'session': 6, 'scan': 14}
['/mnt/BigDisk/2P_scratch/4467975.1/28_09_2020/YMaze_LNovel/YMaze_LNovel_001_003', '/mnt/BigDisk/2P_scratch/4467975.1/28_09_2020/YMaze_LNovel/YMaze_LNovel_006_014']
{}
FOUND BINARIES AND OPS IN ['/mnt/BigDisk/2P_scratch/4467975.1/28_09_2020/combined/suite2p/plane0/ops.npy']
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
NOTE: not running registration, plane already registered
NOTE: applying default /home/mplitt/.suite2p/classifiers/classifier_user.npy
----------- ROI DETECTION
Binning movie in chunks of length 11
Binned movie [3035,416,694], 256.23 sec.
~~~ tried anatomical but failed, install cellpose to use: ~~~
$ pip install cellpose
NOTE: estimated spatial scale ~6 pixels, time epochs 2.53, threshold 18.97 
0 ROIs, score=312.93
1000 ROIs, score=33.71
Found 1645 ROIs, 101.09 sec
After removing overlaps, 1467 ROIs remain
Masks made i

In [12]:
_file_list = ({'date': '04_10_2020', 'scene':'YMaze_LNovel','session':1,'scan':3},
             {'date': '04_10_2020', 'scene':'YMaze_RewardReversal','session':2,'scan':10},)

_h5list,_frs = check_and_make_h5s(_file_list,runh5=False)
ops_dir = os.path.join(write_folder,mouse,"04_10_2020","combined","suite2p","plane0","ops.npy")
ops = np.load(ops_dir, allow_pickle=True).all()
ops['reg_file'] = os.path.join(s2p_dir,"data.bin")
ops['reg_file_chan2'] = os.path.join(s2p_dir,"data_chan2.bin")
np.save(ops_dir,ops)


_h5list,_frs = check_and_make_h5s(_file_list, runh5=False)
ops = TwoPUtils.s2p.set_ops(d={'data_path': _h5list,
                               'save_path0': os.path.join(write_folder,mouse,"04_10_2020","combined"),
                                'fast_disk':[],
                                'move_bin':True,
                                'two_step_registration':True,
                                'maxregshiftNR':10,
                                'nchannels':2,
                                'tau':.7,
                                'functional_chan':1,
                                'fs':_frs[0],
                                'roidetect':True,
                                'input_format':"h5",
                                'h5py_key':'data',
                                'nchannels':2,
                                'threshold_scaling':1.1,
                                'chan2_thresh':.01})
ops=s2p.run_s2p(ops=ops)


{'date': '04_10_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 3}
{'date': '04_10_2020', 'scene': 'YMaze_RewardReversal', 'session': 2, 'scan': 10}
['/mnt/BigDisk/2P_scratch/4467975.1/04_10_2020/YMaze_LNovel/YMaze_LNovel_001_003', '/mnt/BigDisk/2P_scratch/4467975.1/04_10_2020/YMaze_RewardReversal/YMaze_RewardReversal_002_010']
{'date': '04_10_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 3}
{'date': '04_10_2020', 'scene': 'YMaze_RewardReversal', 'session': 2, 'scan': 10}
['/mnt/BigDisk/2P_scratch/4467975.1/04_10_2020/YMaze_LNovel/YMaze_LNovel_001_003', '/mnt/BigDisk/2P_scratch/4467975.1/04_10_2020/YMaze_RewardReversal/YMaze_RewardReversal_002_010']
{}
FOUND BINARIES AND OPS IN ['/mnt/BigDisk/2P_scratch/4467975.1/04_10_2020/combined/suite2p/plane0/ops.npy']
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
NOTE: not running registration, plane already registered
NOTE: applying default /home/mplitt/.suite2p/classifiers/classifier_user.npy
----------- ROI DETECTION
Binni

In [16]:
_file_list = ({'date': '06_10_2020', 'scene':'YMaze_RewardReversal','session':1,'scan':2},
             {'date': '06_10_2020', 'scene':'YMaze_LNovel','session':1,'scan':5})

_h5list,_frs = check_and_make_h5s(_file_list,runh5=False)
ops = TwoPUtils.s2p.set_ops(d={'data_path': _h5list,
                               'save_path0': os.path.join(write_folder,mouse,"06_10_2020","combined"),
                                'fast_disk':[],
                                'move_bin':True,
                                'two_step_registration':True,
                                'maxregshiftNR':10,
                                'nchannels':2,
                                'tau':.7,
                                'functional_chan':1,
                                'fs':_frs[0],
                                'roidetect':True,
                                'input_format':"h5",
                                'h5py_key':'data',
                                'nchannels':2,
                               'threshold_scaling':1.1,
                                'chan2_thresh':.01})
ops=s2p.run_s2p(ops=ops)


{'date': '06_10_2020', 'scene': 'YMaze_RewardReversal', 'session': 1, 'scan': 2}
{'date': '06_10_2020', 'scene': 'YMaze_LNovel', 'session': 1, 'scan': 5}
['/mnt/BigDisk/2P_scratch/4467975.1/06_10_2020/YMaze_RewardReversal/YMaze_RewardReversal_001_002', '/mnt/BigDisk/2P_scratch/4467975.1/06_10_2020/YMaze_LNovel/YMaze_LNovel_001_005']
{}
FOUND BINARIES AND OPS IN ['/mnt/BigDisk/2P_scratch/4467975.1/06_10_2020/combined/suite2p/plane0/ops.npy']
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
NOTE: not running registration, plane already registered
NOTE: applying default /home/mplitt/.suite2p/classifiers/classifier_user.npy
----------- ROI DETECTION
Binning movie in chunks of length 11
Binned movie [3306,416,694], 199.86 sec.
NOTE: estimated spatial scale ~12 pixels, time epochs 2.75, threshold 30.30 
0 ROIs, score=478.44
1000 ROIs, score=40.81
Found 1287 ROIs, 74.90 sec
After removing overlaps, 1212 ROIs remain
Masks made in 27.51 sec.
>>>> CELLPOSE estimating masks in anatomical chan

In [13]:
for h5 in h5list_all:
    !rm {h5}/*.h5 